<a href="https://colab.research.google.com/github/Hafsa620/PAI-LAB/blob/main/Sentiment_Analysis(LAB_12).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-- Sentiment Analysis (Text Classification)
- Downloading Datset from Kaggle to Google Colab
- Text Cleaning
- Text Preprocessing
- Feature Engineering
- ML Model

In [1]:
!pip install kaggle
import os
import json

In [2]:
with open('/content/kaggle (1).json') as file:
  kaggle_json = json.load(file)
  os.environ['KAGGLE_USERNAME'] = kaggle_json['username']
  os.environ['KAGGLE_KEY'] = kaggle_json['key']

In [3]:
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 27% 7.00M/25.7M [00:00<00:00, 71.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 138MB/s] 


In [4]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


Importing Preprocessing Libraries

In [5]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stw=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

Load dataset

In [7]:
data=pd.read_csv('/content/IMDB Dataset.csv')

Text Cleaning

In [8]:
data.shape

(50000, 2)

In [9]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df=data.iloc[:30000]

In [11]:
#Converting Lowercase
df['review']=df['review'].str.lower()

<ipython-input-11-6e26e60aa16a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].str.lower()


In [12]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
29995,"new york, i love you finally makes it to our s...",positive
29996,this movie makes you wish imdb would let you v...,negative
29997,"space camp, which had the unfortunate luck to ...",negative
29998,"octavio paz, mexican poet, writer, and diploma...",positive


In [13]:
#Removing html tags, special characters and extra spaces
def html_tag(text):
  text=re.sub('<.*?>','',text)
  return text

def remove_special_char(text):
  text=re.sub('[^a-zA-Z0-9\s]','',text)
  return text

def remove_extra_space(text):
  text=re.sub('\s+',' ',text)
  return text

In [14]:
df['review']=df['review'].apply(html_tag)

<ipython-input-14-2b24403717bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(html_tag)


In [15]:
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

In [16]:
df['review']=df['review'].apply(remove_special_char)
df['review']=df['review'].apply(remove_extra_space)

<ipython-input-16-f66dd80da2a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_special_char)
<ipython-input-16-f66dd80da2a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_extra_space)


In [17]:
df['review'][1]

'a wonderful little production the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

In [18]:
#tokenization
df['review']=df['review'].apply(word_tokenize)
#stopword removal
df['review']=df['review'].apply(lambda x: [word for word in x if word not in stw])
#lemmatization
df['review']=df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
#join tokens
df['review'] = df['review'].apply(lambda x: ' '.join(x))

<ipython-input-18-4e7fbfa78790>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(word_tokenize)
<ipython-input-18-4e7fbfa78790>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: [word for word in x if word not in stw])
<ipython-input-18-4e7fbfa78790>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

- Feature engineering

In [19]:
#Target column encoding
from sklearn.preprocessing import LabelEncoder

X=df['review']
y=df['sentiment']

le=LabelEncoder()
y=le.fit_transform(y)

print(y)

[1 1 1 ... 0 1 0]


- Bag of words

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

#splitting dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train)

#apply bag of words vectorization
cv=CountVectorizer()
X_train_bow=cv.fit_transform(X_train)
X_test_bow=cv.transform(X_test)

#results of vectorization
print(f'X_train_bow:{X_train_bow.shape}')
print(f'X_test_bow:{X_test_bow.shape}')

#train classifier
rf=RandomForestClassifier(n_estimators=100,random_state=42)
rf.fit(X_train_bow,y_train)

#test classifier
y_pred=rf.predict(X_test_bow)

#evaluate classifier
print(f'Accuracy Score:{accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}')
print(f'Classification Report:\n{classification_report(y_test,y_pred)}')


21753    ever sit movie like one big wtf welcome decoy ...
251      ok bought film woolworth friend joke present b...
22941    hilary great julie pat magnificent mr miyagi r...
618      237 high school student commits suicide shown ...
17090    film reappeared channel 13 1990s series comedy...
                               ...                        
29802    first introduced movie san antonio tx movie 2n...
5390     cant knock film terribly obvious midway watchi...
860      production quite surprise absolutely love obsc...
15795    decent movie although little bit short time pa...
23654    another good mann flick thanks fatherson combi...
Name: review, Length: 24000, dtype: object
X_train_bow:(24000, 130958)
X_test_bow:(6000, 130958)
Accuracy Score:0.8525
Confusion Matrix:
[[2612  417]
 [ 468 2503]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      3029
           1       0.86      0.84      0.85      2971



- Ngram (2)

In [21]:
cv=CountVectorizer(ngram_range=(2,2))
X_train_ngram=cv.fit_transform(X_train)
X_test_ngram=cv.transform(X_test)
rf_ngram=RandomForestClassifier()
rf_ngram.fit(X_train_ngram,y_train)
y_pred_ngram=rf_ngram.predict(X_test_ngram)
print(f'Accuracy Score:{accuracy_score(y_test,y_pred_ngram)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred_ngram)}')
print(f'Classification Report:\n{classification_report(y_test,y_pred_ngram)}')

Accuracy Score:0.7835
Confusion Matrix:
[[2133  896]
 [ 403 2568]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.70      0.77      3029
           1       0.74      0.86      0.80      2971

    accuracy                           0.78      6000
   macro avg       0.79      0.78      0.78      6000
weighted avg       0.79      0.78      0.78      6000



In [22]:
#Results after applying 2 Gram vectorization
print(f'X_train_ngram:{X_train_ngram.shape}')
print(f'X_test_ngram:{X_test_ngram.shape}')

X_train_ngram:(24000, 1702067)
X_test_ngram:(6000, 1702067)


-- TF_IDF

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
X_train_tf=tf.fit_transform(X_train)
X_test_tf=tf.transform(X_test)

#Results after applying tfidf
print(f'X_train_tf:{X_train_tf.shape}')
print(f'X_test_tf:{X_test_tf.shape}')

#Train classifier
rf_tf=RandomForestClassifier()
rf_tf.fit(X_train_tf,y_train)

#Testing
y_pred_tf=rf_tf.predict(X_test_tf)

#Evaluation
print(f'Accuracy Score:{accuracy_score(y_test,y_pred_tf)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred_tf)}')
print(f'Classification Report:\n{classification_report(y_test,y_pred_tf)}')

X_train_tf:(24000, 130958)
X_test_tf:(6000, 130958)
Accuracy Score:0.8533333333333334
Confusion Matrix:
[[2618  411]
 [ 469 2502]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      3029
           1       0.86      0.84      0.85      2971

    accuracy                           0.85      6000
   macro avg       0.85      0.85      0.85      6000
weighted avg       0.85      0.85      0.85      6000

